In [ ]:
import os
import pandas as pd
import clickhouse_connect

In [ ]:
client = clickhouse_connect.get_client(
    host=os.getenv("CLICKHOUSE_HOST"),
    port=int(os.getenv("CLICKHOUSE_PORT")),
    username=os.getenv("CLICKHOUSE_USER"),
    password=os.getenv("CLICKHOUSE_PASSWORD")
)
alert = client.query_df("""
SELECT server_ip, time_window
FROM cem_kpi.debug_alert_p20
WHERE alert_server IN (2,3)
""")
#kết nối đến clickhouse và lấy dữ liệu từ bảng debug_alert_p20 với điều kiện alert_server là 2 hoặc 3, lưu vào dataframe alert

In [17]:
alert.head(21)

,server_ip,time_window
0,101.91.22.249,2025-12-27 08:00:00+07:00
1,115.238.23.77,2025-12-29 18:30:00+07:00
2,115.238.23.77,2025-12-31 03:00:00+07:00
3,115.238.23.77,2025-12-31 09:30:00+07:00
4,115.238.23.78,2025-12-27 20:30:00+07:00
5,115.238.23.78,2025-12-31 03:00:00+07:00
6,115.238.23.78,2025-12-31 09:30:00+07:00
7,143.92.82.17,2025-12-28 18:30:00+07:00
8,158.247.7.204,2025-12-28 20:00:00+07:00
9,158.247.7.204,2025-12-28 23:30:00+07:00


In [ ]:
def get_raw_table(time_window):
    date_str=time_window.strftime('%Y_%d_%m')
    return f"cem_network_test.result_full_flow_{date_str}"
#hàm để lấy dữ liệu thô từ bảng tương ứng với time_window và trả về bảng dữ liệu chứa dataframe tương ứng

In [ ]:
for _,row in alert.iterrows():
    server_ip = row['server_ip']
    time_window = row['time_window']
    table = get_raw_table(time_window)
    start_ts=int(time_window.timestamp())
    end_ts=start_ts+1800
    # câu lệnh query để lấy dữ liệu thô từ ClickHouse theo server_ip và time_window 
    query = f"""
    SELECT
        client_ip,
        client_mac,
        tls_domain,
        dns_domain
    FROM {table}
    WHERE 
        server_ip = '{server_ip}'
        AND from_time>={start_ts}
        AND from_time<{end_ts}
    """
    # Lấy dữ liệu thô từ ClickHouse theo server_ip và time_window với query là ở trên
    raw_df=client.query_df(query) 
    if raw_df.empty: 
        continue
    #lưu dữ liệu vào biến agg từ raw_df với việc nhóm theo client_ip, client_mac, tls_server, dns_server và đếm số lượng flow
    agg=(
        raw_df
        .groupby(['client_ip','client_mac','tls_domain','dns_domain'])
        .size()
        .reset_index(name='total_flow')
    )
    agg['server_ip']=server_ip
    agg['time_window']=time_window
    #lưu dữ liệu vào database 
    client.insert_df('cem_kpi.alert_client_trace', 
    agg [
            [
            'time_window',
            'client_ip',
            'server_ip',
            'client_mac',
            'dns_domain',
            'tls_domain',
            'total_flow'
            ]  
        ]
    )


